In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Importing Cool stuff

In [ ]:
import torch
import torch.functional

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score
import sklearn
import numpy

import keras
from keras.layers import Dense
from keras.models import Sequential
import keras.layers as layers

## Reading Data

In [ ]:
sst1_xt=pd.read_csv("/kaggle/input/dlnlp-assig2/DLNLP assig2/SST-1.x.train.txt",header=None)
sst1_yt=pd.read_csv("/kaggle/input/dlnlp-assig2/DLNLP assig2/SST-1.y.train.txt",header=None)
sst1_xd=pd.read_csv("/kaggle/input/dlnlp-assig2/DLNLP assig2/SST-1.x.dev.txt",header=None)
sst1_yd=pd.read_csv("/kaggle/input/dlnlp-assig2/DLNLP assig2/SST-1.y.dev.txt",header=None)

## Cleaning the corpus using nltk

In [ ]:
tokenizer=nltk.RegexpTokenizer(r"\w+")
def Clean(data):
    for i,j in enumerate(data):
        text=str(j).lower()
        text=re.sub('\w*\d\w*','',text)
        text = re.sub('\[.*?\]', '', text)
        data[i]=tokenizer.tokenize(text)
    return data

sst1_xt[0]=Clean(sst1_xt[0])
sst1_xt

In [ ]:
train_zs="../input/assign-4-labeld/unlabeled_x.txt"

with open(train_zs, 'rb') as f:
        train_zs_lines = f.read().splitlines()
train_zs_lines = [x.decode('ISO-8859-1') for x in train_zs_lines]
a=pd.DataFrame(train_zs_lines)
a[0]=Clean(a[0])
z=[]
for samp in a[0]:
    z.append(samp)

## Lemmatization & stemmatization which one?
Lemmatization i will use 

In [ ]:
def Lemmatization(data):
    lemmatizer=WordNetLemmatizer()
    for i,sentence in enumerate(data):
        data[i]=[lemmatizer.lemmatize(words) for words in sentence]
    
    return data
sst1_xt[0]=Lemmatization(sst1_xt[0])
sst1_xt

## Removing Stopwords

In [ ]:
def StopWords(data):
    stop_words=set(stopwords.words('english'))
    for i,sentence in enumerate(data):
        data[i]=[word for word in sentence if not word in stop_words]
    return data

sst1_xt[0]=StopWords(sst1_xt[0])
sst1_xt.head(10)

## Using glove vectors

In [ ]:
def GloveModel():
    path="../input/nlpword2vecembeddingspretrained/glove.6B.300d.txt"
    file=open(path,"r")
    glovemodel={}
    for line in file:
        splitline=line.split()
        word=splitline[0]
        wordemb=np.array([float(value) for value in splitline[1:]])
        glovemodel[word]=wordemb
    print(f"word_loaded successfully")
    return glovemodel
glovemodel=GloveModel()

In [ ]:
a=0
sst1_xd[0]=Clean(sst1_xd[0])
sst1_xd[0]=Lemmatization(sst1_xd[0])
sst1_xd[0]=StopWords(sst1_xd[0])
for sentence in sst1_xd[0]:
    a=max(a,len(sentence))

## Taking average of words for sentence vectors

In [ ]:
def SentenceVector(data):
    for i,sentence in enumerate(data):
        vector=np.zeros(300)
        for word in sentence:
            if word in glovemodel.keys():
                vector+=glovemodel[word]
        if(len(sentence)>0):
            data[i]=vector/len(sentence)
        else:
            data[i]=vector
    return data

## Converting to sentence matrix

In [ ]:
sentence=np.zeros((sst1_xt.shape[0],300))
for i,j in enumerate(data):
    sentence[i]=j[0:300]

## Sentence matrix for test set

In [ ]:
sst1_xd=pd.read_csv("/kaggle/input/dlnlp-assig2/DLNLP assig2/SST-1.x.dev.txt",header=None)
sst1_xd[0]=Clean(sst1_xd[0])
sst1_xd[0]=Lemmatization(sst1_xd[0])
sst1_xd[0]=StopWords(sst1_xd[0])
sst1_xd[0]=SentenceVector(sst1_xd[0])
sentence1=np.zeros((sst1_xd.shape[0],300))
for i,j in enumerate(sst1_xd[0]):
    sentence1[i]=j[0:300]
sentence1.shape

## SST-2 dataset

In [ ]:
def sst2():
    ## reading data
    sst2_xt=pd.read_csv("/kaggle/input/dlnlp-assig2/DLNLP assig2/SST-2.x.train.txt",header=None)
    sst2_yt=pd.read_csv("/kaggle/input/dlnlp-assig2/DLNLP assig2/SST-2.y.train.txt",header=None)
    sst2_xd=pd.read_csv("/kaggle/input/dlnlp-assig2/DLNLP assig2/SST-2.x.dev.txt",header=None)
    sst2_yd=pd.read_csv("/kaggle/input/dlnlp-assig2/DLNLP assig2/SST-2.y.dev.txt",header=None)
    
    ## cleaning data & getting vectors
    
    # for dev data
    sst2_xd[0]=Clean(sst2_xd[0])
    sst2_xd[0]=Lemmatization(sst2_xd[0])
    sst2_xd[0]=StopWords(sst2_xd[0])
    sst2_xd[0]=SentenceVector(sst2_xd[0])
    sentence1=np.zeros((sst2_xd.shape[0],300))
    
    #for train data
    sst2_xt[0]=Clean(sst2_xt[0])
    sst2_xt[0]=Lemmatization(sst2_xt[0])
    sst2_xt[0]=StopWords(sst2_xt[0])
    sst2_xt[0]=SentenceVector(sst2_xt[0])
    sentence=np.zeros((sst2_xt.shape[0],300))
    
    ## creating vector matrix 
    sentence1=np.zeros((sst2_xd.shape[0],300))
    for i,j in enumerate(sst2_xd[0]):
        sentence1[i]=j[0:300]
    sentence1.shape
    
    sentence=np.zeros((sst2_xt.shape[0],300))
    for i,j in enumerate(sst2_xt[0]):
        sentence[i]=j[0:300]
    sentence.shape
    
    
    
    x_train=sentence
    y_train=pd.get_dummies(sst2_yt)
    x_test=sentence1
    y_test=pd.get_dummies(sst2_yd)

    model=Sequential()
    model.add(Dense(50,input_dim=x_train.shape[1],activation='relu'))
    model.add(Dense(20,activation='relu'))
    model.add(Dense(10,activation='relu'))
    model.add(Dense(y_train.shape[1],activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(x_train,y_train,epochs=14,batch_size=60)
    _,accuracy=model.evaluate(x_train,y_train)

    output=model.predict(x_test)

    result=np.zeros((output.shape[0],output.shape[1]))
    for i,j in enumerate(output):
        result[i][numpy.argmax(output[i])]=1
    accuracy=accuracy_score(y_test,result)
    print(f"test accuracy on sst2 dataset: {accuracy}")
    return model

modelsst2=sst2()


## CR

In [ ]:
def CR():
    ## reading data
    file1 = open('/kaggle/input/dlnlp-assig2/DLNLP assig2/CR.x.train.txt', 'r') 
    lists = file1.readlines()
    sst2_xt=[]
    for i in range(len(lists)):
        sst2_xt.append(lists[i].strip('\n'))
    sst2_xt=pd.DataFrame(sst2_xt)

    file2 = open('/kaggle/input/dlnlp-assig2/DLNLP assig2/CR.y.train.txt', 'r') 
    listsy = file2.readlines()
    sst2_yt=[]
    for i in range(len(listsy)):
        sst2_yt.append(listsy[i].strip('\n'))
    sst2_yt=pd.DataFrame(sst2_yt)
    
    sst2_xt[0]=Clean(sst2_xt[0])
    sst2_xt[0]=Lemmatization(sst2_xt[0])
    sst2_xt[0]=StopWords(sst2_xt[0])
    sst2_xt[0]=SentenceVector(sst2_xt[0])
    sentence=np.zeros((sst2_xt.shape[0],300))
    sst2_xt,sst2_xd,sst2_yt,sst2_yd=sklearn.model_selection.train_test_split(sst2_xt,sst2_yt,test_size=0.001)
    
    ## creating vector matrix 
    sst2_xt=pd.DataFrame(sst2_xt)
    sst2_yt=pd.DataFrame(sst2_yt)
    sst2_xd=pd.DataFrame(sst2_xd)
    sst2_yd=pd.DataFrame(sst2_yd)
    sentence1=np.zeros((sst2_xd.shape[0],300))
    for i,j in enumerate(sst2_xd[0]):
        sentence1[i]=j[0:300]
    sentence1.shape
    
    sentence=np.zeros((sst2_xt.shape[0],300))
    for i,j in enumerate(sst2_xt[0]):
        sentence[i]=j[0:300]
    sentence.shape
    
#     for i,j in range(sst2_yt)
    print(sst2_yt.shape[1])
    x_train=sentence
    y_train=pd.get_dummies(sst2_yt)
    x_test=sentence1
    y_test=pd.get_dummies(sst2_yd)
    print(y_train.shape[1])
    

    model=Sequential()
    model.add(Dense(50,input_dim=x_train.shape[1],activation='relu'))
    model.add(Dense(20,activation='relu'))
    model.add(Dense(10,activation='relu'))
    model.add(Dense(y_train.shape[1],activation='softmax'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    my_callbacks = [keras.callbacks.EarlyStopping(patience=20)]
    model.fit(x_train,y_train,epochs=10,batch_size=50,callbacks=my_callbacks,validation_data=(x_test,y_test))
    _,accuracy=model.evaluate(x_train,y_train)

    output=model.predict(x_test)
    result=np.zeros((output.shape[0],output.shape[1]))
    for i,j in enumerate(output):
        result[i][np.argmax(output[i])]=1
    accuracy=accuracy_score(y_test,result)
    print(f"test accuracy on sst2 dataset: {accuracy}")
       
    file2 = open('../input/dlnlp-assig2/CR.x.test.txt', 'r') 
    listsy = file2.readlines()
    x_gen=[]
    for i in range(len(listsy)):
        x_gen.append(listsy[i].strip('\n'))
    x_gen=pd.DataFrame(x_gen)
    x_gen[0]=Clean(x_gen[0])
    x_gen[0]=Lemmatization(x_gen[0])
    x_gen[0]=StopWords(x_gen[0])
    x_gen[0]=SentenceVector(x_gen[0])

    sentence12=np.zeros((x_gen.shape[0],300))

    for i,j in enumerate(x_gen[0]):
        sentence12[i]=j[0:300]
    x_test1=sentence12

    output=model.predict(x_test1)
    result=np.zeros((output.shape[0],output.shape[1]))
    for i,j in enumerate(output):
        result[i][numpy.argmax(output[i])]=1

    result=pd.DataFrame(result)
    result.columns=y_test.columns
    df = result.copy()
    df['columns'] = df.apply(lambda x: df.columns[x.argmax()][2:], axis=1)
    df['columns'].to_csv("16701.CR.charagram.y.test.txt",header=False,index=False)    
CR()


## MODEL

In [ ]:
print(pd.get_dummies(sst1_yd).head)
pd.get_dummies(sst1_yt)

In [ ]:
x_train=pd.DataFrame(sentence)
y_train=pd.get_dummies(sst1_yt)
x_test=pd.DataFrame(sentence1)
y_test=pd.get_dummies(sst1_yd)

model=Sequential()
model.add(Dense(50,input_dim=x_train.shape[1],activation='relu'))
model.add(Dense(20,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(y_train.shape[1],activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=14,batch_size=50)

output=model.predict(x_test)
result=np.zeros((output.shape[0],output.shape[1]))
for i,j in enumerate(output):
    result[i][np.argmax(output[i])]=1
accuracy=accuracy_score(y_test,result)
print(f"test accuracy on sst1 dataset: {accuracy}")

## Generating output files

In [ ]:
x_gen=pd.read_csv("../input/dlnlp-assig2/CR.x.test.txt",header=None)
x_gen[0]=Clean(x_gen[0])
x_gen[0]=Lemmatization(x_gen[0])
x_gen[0]=StopWords(x_gen[0])
x_gen[0]=SentenceVector(x_gen[0])

sentence12=np.zeros((x_gen.shape[0],300))

for i,j in enumerate(x_gen[0]):
    sentence12[i]=j[0:300]
x_test1=sentence12

output=model.predict(x_test1)
result=np.zeros((output.shape[0],output.shape[1]))
for i,j in enumerate(output):
    result[i][numpy.argmax(output[i])]=1

result=pd.DataFrame(result)
result.columns=y_test.columns
df = result.copy()
df['columns'] = df.apply(lambda x: df.columns[x.argmax()][2:], axis=1)
df['columns'].to_csv("16701.MR.y.test.txt",header=False,index=False)

In [ ]:
y_test.value_counts()

In [ ]:
df = y_test.copy()
df['columns'] = df.apply(lambda x: df.columns[x.argmax()][2:], axis=1)
df

## YOON KIN CNN

## Converting sentence into 2D matrix

In [ ]:
def Sent2DVec(data):
    Sent2D=np.zeros((data.shape[0],30,300))
    for i,sentence in enumerate(data):
        vector=np.zeros((30,200))
        for j,word in enumerate(sentence):
            if word in glovemodel.keys():
                vector[j]=glovemodel[word]
            Sent2D[i]=vector
    return Sent2D

In [ ]:
a=Sent2DVec(sst1_xt[0])
a=np.reshape(a,(a.shape[0],a.shape[1],a.shape[2],1))
a.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D

a=Sent2DVec(sst1_xt[0])
a=np.reshape(a,(a.shape[0],a.shape[1],a.shape[2],1))
x_train=a

y_train=pd.get_dummies(sst1_yt)
sst1_xd=pd.read_csv("/kaggle/input/dlnlp-assig2/DLNLP assig2/SST-1.x.dev.txt",header=None)
sst1_xd[0]=Clean(sst1_xd[0])
sst1_xd[0]=Lemmatization(sst1_xd[0])
sst1_xd[0]=StopWords(sst1_xd[0])
a=Sent2DVec(sst1_xd[0])
a=np.reshape(a,(a.shape[0],a.shape[1],a.shape[2],1))
x_test=a
y_test=pd.get_dummies(sst1_yd)
model=Sequential()
model.add(Conv2D(300,(2,200),activation='relu',input_shape=(30,200,1)))
model.add(layers.MaxPooling2D((29, 1)))
model.add(Dense(20,activation='relu'))
model.add(Dense(5,activation='softmax'))
#callbacks
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=12,batch_size=1,callbacks=[callback],validation_data=(x_test,y_test))

## Testing CNN on test set

In [ ]:
def CnnTest():
    sst1_xd=pd.read_csv("/kaggle/input/dlnlp-assig2/DLNLP assig2/SST-1.x.dev.txt",header=None)
    sst1_xd[0]=Clean(sst1_xd[0])
    sst1_xd[0]=Lemmatization(sst1_xd[0])
    sst1_xd[0]=StopWords(sst1_xd[0])
    a=Sent2DVec(sst1_xd[0])
    a=np.reshape(a,(a.shape[0],a.shape[1],a.shape[2],1))
    x_test=a
    y_test=pd.get_dummies(sst1_yd)
    output=model.predict(x_test)
    output=np.reshape(output,(output.shape[0],output.shape[3]))
    result=np.zeros((output.shape[0],output.shape[1]))
    for i,j in enumerate(output):
        result[i][np.argmax(output[i])]=1
    accuracy=accuracy_score(y_test,result)
    print(f"test accuracy on sst1 dataset: {accuracy}")
    return x_test.shape
    
CnnTest()

In [ ]:
model.summary()